# Chicago Car Crashes Analysis (2021-2023)
## Phase 3 Project - Classification Model

## 1. Environment Setup

In [2]:
# Install required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')

print("All packages imported successfully!")


All packages imported successfully!


## 2. Data Loading & Initial Inspection

In [3]:
# Load dataset 
try:
    df = pd.read_csv('car-crashes-data-2021-2023.csv')
    print(f"Dataset loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns")
except FileNotFoundError:
    print("ERROR: File not found. Please check the file path.")


Dataset loaded successfully with 327608 rows and 48 columns


In [4]:
# Initial inspection
if 'df' in locals():
    print("\nFirst 3 rows:")
    display(df.head(3))
    
    print("\nBasic info:")
    df.info()
    
    print("\nMissing values:")
    display(df.isnull().sum().sort_values(ascending=False).head(10))


First 3 rows:


,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,397c3f7cdef230e87e05c42cbfa2081ebd5f1e019fe0da...,NaN,12/31/2023 02:16:00 PM,35,NO CONTROLS,NO CONTROLS,SNOW,DAYLIGHT,TURNING,ONE-WAY,...,0.0,0.0,2.0,0.0,14,1,12,41.864985,-87.619511,POINT (-87.6195106093 41.864984899958)
1,45bb4ab28f23c5670954845b18b7d8c818fe6cc3b57201...,NaN,12/31/2023 02:10:00 PM,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,REAR TO REAR,PARKING LOT,...,0.0,0.0,2.0,0.0,14,1,12,41.979836,-87.807127,POINT (-87.807127417247 41.979836274251)
2,dbbe9ec79f44275a97c327f07b3e9e1f33f78921456f24...,NaN,12/31/2023 02:05:00 PM,30,NO CONTROLS,NO CONTROLS,SNOW,DAYLIGHT,TURNING,NOT DIVIDED,...,0.0,0.0,5.0,0.0,14,1,12,41.926800,-87.680218,POINT (-87.680217585739 41.92679963502)



Basic info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327608 entries, 0 to 327607
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                327608 non-null  object 
 1   CRASH_DATE_EST_I               24730 non-null   object 
 2   CRASH_DATE                     327608 non-null  object 
 3   POSTED_SPEED_LIMIT             327608 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         327608 non-null  object 
 5   DEVICE_CONDITION               327608 non-null  object 
 6   WEATHER_CONDITION              327608 non-null  object 
 7   LIGHTING_CONDITION             327608 non-null  object 
 8   FIRST_CRASH_TYPE               327608 non-null  object 
 9   TRAFFICWAY_TYPE                327608 non-null  object 
 10  LANE_CNT                       41 non-null      float64
 11  ALIGNMENT                      327608 non-null  object 
 12  ROADWAY_SURFACE_C

LANE_CNT                  327567
WORKERS_PRESENT_I         327163
DOORING_I                 326675
WORK_ZONE_TYPE            326464
WORK_ZONE_I               326080
PHOTOS_TAKEN_I            323133
STATEMENTS_TAKEN_I        319438
NOT_RIGHT_OF_WAY_I        312787
CRASH_DATE_EST_I          302878
INTERSECTION_RELATED_I    250951
dtype: int64